# 📚 기말고사 범위 (Week 9-12)
## 주제: 해시 테이블 (Hash Table) - 이론 및 구현

# 해시 테이블 구현하기

In [ ]:
'''
	해시 테이블: 알고리즘 구현(Python) -- 개방형 주소 방식
		파일명: HashOpenAddressing.py
        	- __main__: 데이터 삽입.삭제.검색, 전체 출력
		- 클래스: HashOpenAddressing
			데이터 삽입: insert
			데이터 삭제: delete
			데이터 검색: search
			전체 출력: output
'''

# 클래스 설계: HashOpenAddressing
class HashOpenAddressing: 
    def __init__(self, n:int):
        self.__table = [ None for i in range(n) ]
        self.__count = 0
        self.__DELETED = -54321

    # 해시 함수
    def __Hash(self, i:int, num) -> int:
        return (num + i) % len(self.__table)        # 선형 조사
        # return (num + i*i) % len(self.__table)    # 이차 조사

    # 데이터 삽입.삭제.검색
    def insert(self, num):
        # 1. 테이블이 가득 찼는지 확인
        if self.__count >= len(self.__table):
            print("해시 테이블이 가득 찼습니다!")
            return None
        '''
        #2. 이미 존재하는 값인지 확인 (선택사항)
        if self.search(num):
            print(f"{num}은(는) 이미 존재합니다 !")
            return None
        '''
        # 3. 삽입 시도
        for i in range(len(self.__table)):
            slot = self.__Hash(i, num)
            # \(백슬래시) : 줄 바꿈 연속 기호 : Python에서 긴 코드를 여러 줄로 나눌 때 사용하는 기호
            if self.__table[slot] == None \
            or self.__table[slot] == self.__DELETED:
                self.__table[slot] = num
                self.__count += 1
                return num

        return None
    
    def delete(self, num):
        i = 0
        while i < len(self.__table):
            index = self.__Hash(i, num)

            if self.__table[index] == num:
                self.__table[index] = self.__DELETED
                self.__count -= 1
                print(f'{num} 삭제 성공 !!')
                return True
            
            elif self.__table[index] == None:
                print(f' {num} 은(는)해시 테이블에 존재하지 않습니다. 다시 입력하십시오.')
                return False

            else:
                i += 1  # 그 자리가 __DELETED면 다음 칸도 검사함.

        return None

    def search(self, num):
        i = 0

        while i < len(self.__table):
            index = self.__Hash(i, num)

            if self.__table[index] == num:
                print(f'{num}은 해시 테이블의 index(0~{len(self.__table)-1}): {index}에 존재합니다 !')
                return num
            
            elif self.__table[index] == None:
                return None
            
            else:
                i += 1

        return None

    # 전체 출력
    def output(self) -> None:
        print(f'count({self.__count}): {self.__table}')

if __name__ == '__main__':
    import os		# system
    import sys  	# exit

    h = HashOpenAddressing(13)
    while (True):
        os.system('clear')
        print('\n ### 해시 테이블 ###')
        print('1) 데이터 삽입')
        print('2) 데이터 삭제')
        print('3) 데이터 검색')
        print('4) 전체 출력')
        print('5) 프로그램 종료\n')
        choice = int(input('메뉴 선택: '))

        match choice:
            case 1:
                while True:
                    print(f'\n현재 저장된 데이터: {h._HashOpenAddressing__count}/13')
                    num = int(input('임의의 정수 입력(종료: 0): '))
                    if num == 0: break
                    
                    result = h.insert(num)
                    if result:
                        print(f'✅ {result} 삽입 성공!')
                    else:
                        break

            case 2:
                while True:
                    num = int(input('임의의 정수 입력(종료: 0): '))
                    if num == 0: break
                    h.delete(num)
            case 3:
                while True:
                    num = int(input('임의의 정수 입력(종료: 0): '))
                    if num == 0: break
                    searched = h.search(num)
                    if searched == None:
                        print(f'{num}: 데이터가 존재하지 않습니다 !')
                        
            case 4: h.output()
            case 5:
                sys.exit("\n프로그램 종료!!!")
            case _: print('\n잘못 선택 하셨습니다. \n')
        input("계속하려면 엔터키를 누르십시오.")

    # del h
    # h.__del__

# 페쇄 주소 방식으로 충돌을 해결하는 해시 테이블 구현하기

In [ ]:
# Closed Addressing Methods

class Chaining:
    class SNode:    # 연결 리스트의 노드
        def __init__(self, data):
            self.data= data
            self.link= None

    def __init__ (self, n: int):
        self.__table = [None] * n
        self.__count = 0    # <- 데이터 개수 추적
        self.__MAX_LOAD_FACTOR = 1.5    # <- 임계값

    def __Hash(self, num):
        return num % len(self.__table)
    
    def __get_load_factor(self):
        """현재 로드 팩터 계산"""
        return self.__count / len(self.__table)
    
    def __resize(self):
        """테이블 크기 2배로 확장"""
        print(f'로드 팩터 {self.__get_load_factor():.2f} -> 테이블 확장 !')
        
        old_table = self.__table
        self.__table = [None] * (len(old_table) * 2)
        self.__count = 0

        # 모든 데이터 재삽입 (새로운 해시값으로 !!)
        for slot in old_table:
            tNode = slot
            while tNode:
                self.insert(tNode.data) # 재삽입
                tNode = tNode.link

    def insert(self, num):
        # 0. 로드 팩터 체크
        if self.__get_load_factor() >= self.__MAX_LOAD_FACTOR:
            self.__resize()

        # 1. 해시 함수로 slot 계산
        slot = self.__Hash(num)

        # Case 1: table[slot]이 None (비어있음)
        if self.__table[slot] == None:
            self.__table[slot] = self.SNode(num)
        
        # Case 2: table[slot]에 이미 노드가 있음 (충돌!)
        else:
            tNode = self.__table[slot]
            while tNode.link:
                tNode = tNode.link
            tNode.link = self.SNode(num)

        self.__count += 1
        return num
    
    def search(self, num):
        # 1. 해시 함수로 slot 계산
        slot = self.__Hash(num)
        
        # 2. table[slot]부터 시작
        tNode = self.__table[slot]

        # 3. 연결 리스트 순회하며 찾기
        while tNode:
            if tNode.data == num:
                return num
            tNode = tNode.link

        return None
    
    def delete(self, num):
        slot = self.__Hash(num)
        tNode = self.__table[slot]

        pre = None

        while tNode and tNode.data != num:
            pre = tNode
            tNode = tNode.link
        
        if tNode == None:
            return None
        
        if pre == None:
            self.__table[slot] = tNode.link
        
        else:
            pre.link = tNode.link

        self.__count -= 1
        data = tNode.data
        del tNode
        return data

    def output(self):
        print(f'\n현재 상태: {self.__count}개 / {len(self.__table)} slots')
        print(f'로드 팩터: {self.__get_load_factor():.2f}\n')

        for i in range(len(self.__table)):
            print(f'{i:3}', end='')

            tNode = self.__table[i]
            count = 0
            while tNode:
                print(f' --> {tNode.data}', end='')
                count += 1
                tNode = tNode.link
            
            if count > 3:
                print(f'  ⚠️ ({count}개)', end='')
            
            print('')

if __name__ == '__main__':       
    h = Chaining(13)

    import os
    import sys

    while (True):
        os.system('clear')
        print('\n ### 해시 테이블 ###')
        print('1) 데이터 삽입')
        print('2) 데이터 삭제')
        print('3) 데이터 검색')
        print('4) 전체 출력')
        print('5) 프로그램 종료\n')
        print('메뉴 선택 : ', end=' ')
        choice = int(input())

        if choice == 1:
            while True:
                print(f'입력된 데이터의 개수: {h._Chaining__count}')
                print(f'로드 팩터: {h._Chaining__get_load_factor():.2f}')
                num = int(input('삽입 할 데이터 입력(종료: 0): '))
                if num == 0:
                    break
                inserted = h.insert(num)
                if inserted: print(f'삽입 된 데이터: {inserted}')
                else: print(f'데이터 삽입에 실패하였습니다.')

        elif choice == 2:
            while True:
                num = int(input('삭제 할 데이터 입력(종료: 0): '))
                if num == 0:
                    break
                deleted = h.delete(num)
                if deleted: print(f'삭제 된 데이터: {deleted}')
                else: print(f'데이터가 존재하지 않습니다.')

        elif choice == 3:
            while True:
                num = int(input('검색 할 데이터 입력(종료: 0): '))
                if num == 0:
                    break
                searched = h.search(num)
                if searched: print(f'검색된 데이터: {h._Chaining__Hash(num)} 자리에: {searched}')
                else: print(f'데이터가 존재하지 않습니다.')

        elif choice == 4: h.output()

        elif choice == 5:
            print('프로그램 종료!!!')
            break
        else:
            print('잘못 선택 하셨습니다!')
        
        input("계속 하려면 엔터를 입력하십시오")
        